In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("Property_stolen_and_recovered.csv")
df

# EDA

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

key observation here
1.   mean is more than median in all columns
2. there is a huge diffence in 75% and max values .. 
these two obseravtion suggests there are outliers inn the dataset

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
# here data  set has no mising values 
sns.heatmap(df.isnull())

Checking the unique values below variables are having . So as to convert the nominal values to numerical for further data modelling

In [ ]:
df.Area_Name.unique()

In [ ]:
df.Group_Name.unique()

In [ ]:
df.Sub_Group_Name.unique()

In [ ]:
df.Year.unique()

In [ ]:
df.head()

In [ ]:
dfcor=df.corr()
dfcor

In [ ]:
plt.Figure(figsize=(6,4))
sns.heatmap(dfcor,annot=True)

All Columns have a positive correlation with taget variable other than year

Year variable is negatively correlated with all columns. we may drop this variable later if reqd 

In [ ]:
#Univariate analysis

In [ ]:
sns.displot(df["Cases_Property_Stolen"])
plt.show()

In [ ]:
sns.displot(df["Cases_Property_Recovered"])
plt.show()

In [ ]:
sns.displot(df["Value_of_Property_Recovered"])
plt.show()

In [ ]:
sns.displot(df["Value_of_Property_Stolen"])
plt.show()

univariate analysis shows the data is highly +vely skewed

In [ ]:
#bivariate analysis

In [ ]:
plt.scatter(df["Value_of_Property_Stolen"],df["Value_of_Property_Recovered"])
plt.xlabel("Value_of_Property_Recovered")
plt.ylabel("Value_of_Property_Stolen")
plt.show()

In [ ]:

plt.scatter(df["Cases_Property_Recovered"],df["Cases_Property_Stolen"])
plt.xlabel("Cases_Property_Recovered")
plt.ylabel("Cases_Property_Stolen")
plt.show()

Bivariate shows the positive correlation in cases variables only

In [ ]:
#multivariate analysis
sns.pairplot(df)

here also we can see that only cases variable are having linear relationship. others are positively skewed or scattered

#transforming the categorical variables to numerical for data modelling using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()


In [ ]:
list1=['Area_Name','Year','Group_Name','Sub_Group_Name']
for i in list1:
    df[i]=le.fit_transform(df[i].astype(str))

In [ ]:
df.head(2)

# handling Outliers

In [ ]:
df["Cases_Property_Recovered"].plot.box()

In [ ]:
df["Cases_Property_Stolen"].plot.box()

In [ ]:
df["Value_of_Property_Recovered"].plot.box()

In [ ]:
df["Value_of_Property_Stolen"].plot.box()

As the property value may different largely from place to place . so outliers shall be present beacause of property prices. so removing outliers in this case will be a big loss of data.  

In [ ]:
df.skew()

In [ ]:
# dividing the x varaibles features from y variable target
#x=df.drop(columns=["Value_of_Property_Recovered"])
#y=df[["Value_of_Property_Recovered"]]
x=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import power_transform

In [ ]:
x=power_transform(x,method='yeo-johnson')

In [ ]:
# scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
scaledX=sc.fit_transform(x)

#finding the best random state

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


In [ ]:
best_rstate=0
accuracy=0
for i in range(30,200):
    x_train,x_test,y_train,y_test=train_test_split(scaledX,y,test_size=.25,random_state=i)
    mod=LinearRegression()
    mod.fit(x_train,y_train)
    predlr=mod.predict(x_test)
    tempaccu=r2_score(y_test,predlr)
    if(tempaccu>accuracy):
        accuracy=tempaccu
        best_rstate=i
        
print("Best Accuracy",accuracy*100, "Random state",best_rstate)        

In [ ]:
# using the best random state

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(scaledX,y,test_size=.25,random_state=149)

# finding the best model

Linear Regression

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
predlr=lr.predict(x_test)
r2score=r2_score(y_test,predlr)
cvscore=cross_val_score(LinearRegression(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


DecisionTreeRegressor

In [ ]:
dtc=DecisionTreeRegressor()
dtc.fit(x_train,y_train)
preddtc=dtc.predict(x_test)
r2score=r2_score(y_test,preddtc)
cvscore=cross_val_score(DecisionTreeRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

RandomForestRegressor

In [ ]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train)
predrf=rf.predict(x_test)
r2score=r2_score(y_test,predrf)
cvscore=cross_val_score(RandomForestRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

GradientBoostingRegressor

In [ ]:
gb=GradientBoostingRegressor()
gb.fit(x_train,y_train)
predgb=gb.predict(x_test)
r2score=r2_score(y_test,predgb)
cvscore=cross_val_score(GradientBoostingRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

# conclusion

RandomForestRegressor is best performing with accuracy 77%



In [ ]:
a=np.array(y_test)
a

In [ ]:
df_con=pd.DataFrame({'original':a,"predicted":predrf},index=range(len(a)))
df_con

In [ ]:
# saving the prediction model

import pickle
filename="Property_stolen_and_recoveredData.pkl"
pickle.dump(predrf,open(filename,'wb'))